In [1]:
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib

# Figure SVG beautification
def init_plot_rc_params(style='whitegrid',font_scale=1.4):
    # Important for AdobeIllustrator import
    sns.reset_defaults()
    plt.rcParams['pdf.fonttype'] = 42
    plt.rcParams['svg.fonttype'] = 'none'
    plt.rcParams['ps.fonttype'] = 42
    plt.rcParams['font.family'] = 'Calibri'
    plt.rcParams['figure.dpi'] = 100
    # Some tweaks for font clarity in figures
    sns.set_style(style)
    sns.set_context('paper', font_scale=font_scale, rc={'xtick.labelsize':14,'ytick.labelsize':14,"font.size":14,"axes.titlesize":13,
                                 "axes.labelsize":14,
        'pdf.fonttype':42,'ps.fonttype':42,'font.family':'Calibri',})
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['svg.fonttype'] = 'none'
    matplotlib.rcParams['ps.fonttype'] = 42
    matplotlib.rcParams['font.family'] = 'Calibri'
    matplotlib.rcParams['figure.dpi'] = 100
init_plot_rc_params()

In [4]:

# Import libraries located in relation to this file
import inspect, os, sys
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
tools_dir = os.path.join(parent_dir, '../lib/')
print "tools_dir", tools_dir
sys.path.insert(0, tools_dir)

from pankoff_lib import *
from vienna_lib import *
from RNAStrand_lib import *


from RNAStrand_lib import parse_strand_dp_file                                
# parsed_dict = parse_strand_dp_file('./l30-300/download-dp-10231.dp')    
parsed_dict = parse_strand_dp_file('./data/RNAstrand-download-dp-10225.dp')    

import pandas as pd
df_all = pd.DataFrame.from_dict(parsed_dict, orient='index')
df_noncrossings = df_all[df_all['pseudoknot']==False]
print len(df_noncrossings), " out of ", len(df_all), " are pseudoknot free"
df_tRNAs = df_noncrossings[df_noncrossings['desc'].str.contains('TRNA')] #[[df_noncrossings.desc]]
print len(df_tRNAs), " out of ", len(df_all), " are tRNAs"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tools_dir /media/milad/ubuntu14/home/milad/workspace/pankoff/LocARNA-Pankoff/../lib/
ASE_00048.dp
*******
ASE_00051.dp
*******
ASE_00251.dp
*******
CRW_00548.dp
*******
CRW_00549.dp
*******
CRW_00550.dp
*******
CRW_00552.dp
*******
CRW_00553.dp
*******
CRW_00554.dp
*******
CRW_00555.dp
*******
CRW_00556.dp
*******
CRW_00557.dp
*******
CRW_00558.dp
*******
CRW_00559.dp
*******
CRW_00560.dp
*******
CRW_00561.dp
*******
CRW_00562.dp
*******
CRW_00563.dp
*******
CRW_00564.dp
*******
CRW_00565.dp
*******
CRW_00566.dp
*******
CRW_00567.dp
*******
CRW_00568.dp
*******
CRW_00569.dp
*******
CRW_00570.dp
*******
CRW_01482.dp
*******
CRW_01484.dp
*******
CRW_01485.dp
*******
CRW_01486.dp
*******
CRW_01487.dp
*******
CRW_01488.dp
*******
CRW_01489.dp
*******
CRW_01490.dp
*******
CRW_01491.dp
*******
CRW_01492.dp
*******
CRW_01493.dp
*******
CRW_01494.dp
*******
CRW_01495.dp
*******
CRW_01496.dp
*******
CRW_0149

SPR_00434.dp   skipped
ERROR: DP strand file line cannot be parsed and skipped: SPR_00435.dp GCCGGUGUAGCUCAAUDGGDAGAGCAACUGACU5GU6APCAGUAG7UUGG
ERROR: DP strand file line cannot be parsed and skipped: SPR_00435.dp GGGTPCAAGUCCUCUUGCCGGCACCA
SPR_00435.dp   skipped
ERROR: DP strand file line cannot be parsed and skipped: SPR_00436.dp GCUGAUAUAGCUCAGDDGGDAGAGCGCACCCUUGGUEAGGGUGAG7UCGG
ERROR: DP strand file line cannot be parsed and skipped: SPR_00436.dp CAGTPCGAAUCUGCCUAUCAGCACCA
SPR_00436.dp   skipped
ERROR: DP strand file line cannot be parsed and skipped: SPR_00437.dp GCUGAUAUGGCUCAGDDGGDAGAGCGCACCCUUGGUEAGGGUGAG7UCCC
ERROR: DP strand file line cannot be parsed and skipped: SPR_00437.dp AGTPCGACUCUGGGUAUCAGCACCA
SPR_00437.dp   skipped
ERROR: DP strand file line cannot be parsed and skipped: SPR_00438.dp GCCCCUUUAACUCAGU#GDAGAGUAACGCCAUGGUEAGGCGUAA7DCAUC
ERROR: DP strand file line cannot be parsed and skipped: SPR_00438.dp GGTPCAAAUCCGAUAAGGGGCUCCA
SPR_00438.dp   skipped
ERROR: DP stran

In [9]:
def my_eval(df,bonus=1.5):
    pankoff_scores, pankoff_min_scores, pankoff_exact_scores, pmcomp_scores, pmcomp_exact_scores = dict(), dict(), dict(), dict(), dict()
    pankoff_corrs, pankoff_min_corrs, pankoff_exact_corrs, pmcomp_corrs, pmcomp_exact_corrs = dict(), dict(), dict(), dict(), dict()
    RNAevals, eval_errors = dict(), dict()
    MFE_structs, MFE_energies, fold_errors = dict(), dict(), dict()
    pankoff_scores = dict()
    count = 0
    newdf = pd.DataFrame()
    eval_ref_struct = False
    for index, row in df.iterrows():
        
        print "*********", count
        count += 1
        print index
    
        ##############################
        # Strand struct evaluation
        
        if eval_ref_struct:
            eval_energy, has_err = eval_rna_structure(row['seq'], row['struct'])
#         print eval_energy, has_err
            RNAevals[index] = eval_energy
            eval_errors[index] = has_err
        
        ##############################
        # RNAsubopt evaluation
        #print index #, row['seq']
        subopt_file = call_RNAsubopt(row['seq'], index, deltaEnergy=5, use_cache=True)
        print subopt_file
        suboptimals = parse_subpot_out(subopt_file, 500)
        #os.remove(subopt_file)
        df_subopts_scores, pankoff_scores[index], pankoff_min_scores[index], pankoff_exact_scores[index], pmcomp_scores[index], pmcomp_exact_scores[index],  MFE_structs[index], MFE_energies[index] =\
            get_all_pmcomp_pankoff_scores(row['seq'], suboptimals, index, t1='1e-5', t2='1e-8',bonus=bonus)
        df_corr_subopts = get_subopt_correlations(df_subopts_scores, selection=True)
        print df_corr_subopts
        pankoff_corrs[index], pankoff_min_corrs[index], pankoff_exact_corrs[index], pmcomp_corrs[index], pmcomp_exact_corrs[index] =\
            df_corr_subopts.loc['rnasubopt-rcorr'][
                ['pankoff', 'pankoff-min', 'pankoff-exact', 'pmcomp', 'pmcomp-exact']].tolist()
        
    if eval_ref_struct:
        newdf['RNAeval-energy'] = pd.Series(RNAevals)
        newdf['eval-error'] = pd.Series(eval_errors)
        print len(newdf[newdf['eval-error']==True]), " out of ", len(newdf), " have RNAeval error"
    newdf['MFE-struct'] = pd.Series(MFE_structs)
    newdf['MFE-energy'] = pd.Series(MFE_energies)
    newdf['fold-error'] = pd.Series(fold_errors)
    newdf['MFE-pankoff'] = pd.Series(pankoff_scores)
    newdf['MFE-pankoff-min'] = pd.Series(pankoff_min_scores)
    newdf['MFE-pankoff-exact'] = pd.Series(pankoff_exact_scores)
    newdf['MFE-pmcomp'] = pd.Series(pmcomp_scores)
    newdf['MFE-pmcomp-exact'] = pd.Series(pmcomp_exact_scores)
    
    newdf['corr-pankoff'] = pd.Series(pankoff_corrs)
    newdf['corr-pankoff-min'] = pd.Series(pankoff_min_corrs)
    newdf['corr-pankoff-exact'] = pd.Series(pankoff_exact_corrs)
    newdf['corr-pmcomp'] = pd.Series(pmcomp_corrs)
    newdf['corr-pmcomp-exact'] = pd.Series(pmcomp_exact_corrs)
    print(newdf.columns)
    
    print len(newdf[newdf['fold-error']==True]), " out of ", len(newdf), " have RNAfold error" 
    

    return df.merge(newdf,left_index=True,right_index=True)


dict_ndfs = {}
precomputed = True

if precomputed:
    dict_ndfs = {}
    for bon in [1.5]:
        dict_ndfs[bon] =  pd.read_csv('./data/20190502-ndfall-l30-200-bonus{}.csv'.format(bon))
    dict_ndfs.keys()
else:
    
    for my_bon in [1.5]:
        my_ndf = my_eval(df_all[:500].copy(), bonus=my_bon)
        dict_ndfs[my_bon] = my_ndf

    for bon in dict_ndfs.keys():
        dict_ndfs[bon].to_csv('./data/201905X-ndfall-l30-200-bonus{}.csv'.format(bon))
    dict_ndfs.keys()

In [10]:
%matplotlib inline 
def plot_violin_RNAstrand(df, use_violin = True, title_suffix=''):
    rename_dict = {'corr-pmcomp':'PMcomp\nenergy',
                    'corr-pmcomp-exact':'PMcomp score\n'+r'$(P_{min}=1/(2\cdot$seq-len$))$',
                    'corr-pankoff':'Pankov\nenergy\n(ML-prod)',
                 'corr-pankoff-min':'Pankov\nenergy\n(ML-min)',
                   'corr-pankoff-exact':'Pankov score\n'+r'(ML-prod,$\beta=1.5$)',
                  'RNAsubopt':'Referenece energy'}

    sns.reset_defaults()

    sns.set_style('darkgrid')
    fancy_pal = sns.color_palette('Paired',12)[:2]+sns.color_palette('Paired',12)[9:10]+sns.color_palette('Paired',12)[4:6]
    fancy_pal2 =  sns.color_palette('colorblind',10)[9:10] + sns.color_palette('colorblind',10)[0:1]+sns.color_palette('colorblind',10)[3:4]+ sns.color_palette('colorblind',10)[1:2]+sns.color_palette('colorblind',10)[5:6] 
    sns.set_palette(fancy_pal2)

    fig, ax = plt.subplots(figsize=(6.5,3))

    vertical = True
    if use_violin:
        sns.violinplot(data=df[['corr-pmcomp','corr-pmcomp-exact','corr-pankoff-min','corr-pankoff','corr-pankoff-exact', ]].rename(rename_dict,axis=1)
                       ,orient="v" if vertical else "h",ax=ax,cut=0,bw=0.2,
        #                color=sns.color_palette('colorblind'),
               )
    else:
        sns.boxplot(data=df[['corr-pankoff','corr-pankoff-min','corr-pankoff-exact', 'corr-pmcomp-exact','corr-pmcomp',]].rename(rename_dict,axis=1)
                       ,orient="v" if vertical else "h",ax=ax,)



    
    if vertical:
        
        ax.xaxis.tick_top()
        ax.set_ylim([-1.05,1.05])
        ax.set_ylabel('Correlations with the reference energies')
        ax.set_xticklabels(ax.get_xticklabels(),multialignment='center')
    else:
        ax.spines['right'].set_visible(False)
        # ax.spines['top'].set_visible(False)
        ax.set_xlim([-1.05,1.05])
        ax.set_xlabel('Correlations with the reference energies')
        ax.yaxis.tick_right()
        ax.set_yticklabels(ax.get_yticklabels(),multialignment='center')
    figname = './figure-violin-RNAstrand-t500{}.svg'.format(title_suffix.replace(' ','-'))
    fig.savefig(figname,bbox_inches='tight')
    print("Figure saved to:", figname)

plot_violin_RNAstrand(dict_ndfs[1.5])

('Figure saved to:', './figure-violin-RNAstrand-t500.svg')
